# Import some packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from OOPAO.tools.displayTools import displayMap
plt.ion()

# Set the number of sub-apertures (WFS sampling)

In [ ]:
n_subaperture = 20

# Telescope


In [ ]:
from OOPAO.Telescope import Telescope

# Create the Telescope object
tel = Telescope(resolution           = 6*n_subaperture,     # Resolution of the telescope in [pix]
                diameter             = 1.52,                # Diameter in [m]
                samplingTime         = 1/1000,              # Sampling time in [s] of the AO loop
                centralObstruction   = 0.25,                # Central obstruction in [%] of a diameter
                display_optical_path = False,               # Flag to display optical path
                fov                  = 10 )                 # Field of view in [arcsec]. If set to 0 (default) this speeds up the computation of the phase screens but is uncompatible with off-axis targets

# Display current pupil
plt.figure()
plt.imshow(tel.pupil)


# Sources


In [ ]:
from OOPAO.Source import Source

# Create the Natural Guide Star object
ngs = Source(optBand     = 'I',           # Optical band (see photometry.py)
             magnitude   = 8,             # Source Magnitude
             coordinates = [0,0])         # Source coordinated [arcsec,deg]

# Combine the NGS with the telescope using '*':
ngs*tel

# Create the Scientific Target object
src = Source(optBand     = 'K',           # Optical band (see photometry.py)
             magnitude   = 8,             # Source Magnitude
             coordinates = [0,0])         # Source coordinated [arcsec,deg]

# combine the SRC to the telescope using '*'
src*tel

# check that the ngs and tel.src objects are the same
tel.src.print_properties()

# The Telescope has an OPD property (Optical Path Difference in [m] that is not wavelength dependant)
plt.figure()
plt.imshow(tel.OPD)
plt.title('Telescope OPD in [m]')

# The Telescope now has a source attached to it with a phase property (in [rad]) that is wavelength dependant:
plt.figure()
plt.imshow(tel.src.phase)
plt.title('Telescope Source Phase in [rad]')

# Compute the PSF

In [ ]:
# Compute the PSF using the computePSF method
ngs*tel
tel.computePSF(zeroPaddingFactor = 6)
log_PSF = np.log10(np.abs(tel.PSF))

# Plot the PSF
plt.figure()
plt.imshow(log_PSF,extent = [tel.xPSF_arcsec[0],tel.xPSF_arcsec[1],tel.xPSF_arcsec[0],tel.xPSF_arcsec[1]])
plt.clim([log_PSF.max()-5, log_PSF.max()])
plt.xlabel('[Arcsec]')
plt.ylabel('[Arcsec]')
plt.colorbar()
plt.title('Log Scale PSF @'+str(tel.src.wavelength*1e9)+' nm')

# Atmosphere

In [ ]:
# Creating the Atmosphere object

from OOPAO.Atmosphere import Atmosphere

atm = Atmosphere(telescope     = tel,                               # Telescope
                 r0            = 0.05,                              # Fried Parameter [m]
                 L0            = 25,                                # Outer Scale [m]
                 fractionalR0  = [0.45 ,0.1  ,0.1  ,0.25  ,0.1   ], # Cn2 Profile
                 windSpeed     = [10   ,12   ,11   ,15    ,20    ], # Wind Speed in [m]
                 windDirection = [0    ,72   ,144  ,216   ,288   ], # Wind Direction in [degrees]
                 altitude      = [0    ,1000 ,5000 ,10000 ,12000 ]) # Altitude Layers in [m]

# Initialize atmosphere with current Telescope
atm.initializeAtmosphere(tel)

# The phase screen can be updated using atm.update method (temporal sampling given by tel.samplingTime)
atm.update()

# Display the atm.OPD = resulting OPD
plt.figure()
plt.imshow(atm.OPD*1e9)
plt.title('OPD Turbulence [nm]')
plt.colorbar()

# Propagate the light through the atmosphere

In [ ]:
# The Telescope and Atmosphere can be combined using the '+' operator (Propagation through the atmosphere):
tel+atm # This operations makes that the tel.OPD is automatically over-written by the value of atm.OPD when atm.OPD is updated.

# It is possible to print the optical path:
tel.print_optical_path()

# display the atm.OPD
plt.figure()
plt.imshow(atm.OPD*1e9)
plt.title('OPD Atmosphere [nm]')
plt.colorbar()

plt.figure()
plt.imshow(tel.OPD*1e9)
plt.title('OPD Telescope [nm]')
plt.colorbar()

plt.figure()
plt.imshow(tel.src.phase)
plt.title('Telescope Source Phase [rad]')
plt.colorbar()

In [ ]:
# display the atmosphere layers for the sources specified in list_src:
atm.display_atm_layers(list_src=[ngs,src])

In [ ]:
# Seeing limited PSFs
# Compute the seeing limited PSF (through the atmosphere) using the computePSF method (for the ngs)
atm*ngs*tel
tel.computePSF(zeroPaddingFactor = 6)

log_PSF = np.log10(np.abs(tel.PSF))

plt.figure()
plt.imshow(log_PSF,extent = [tel.xPSF_arcsec[0],tel.xPSF_arcsec[1],tel.xPSF_arcsec[0],tel.xPSF_arcsec[1]])
plt.clim([log_PSF.max()-5, log_PSF.max()])
plt.xlabel('[Arcsec]')
plt.ylabel('[Arcsec]')
plt.colorbar()
plt.title('Log Scale PSF @'+str(tel.src.wavelength*1e9)+' nm')


# Deformable Mirror

In [ ]:
from OOPAO.DeformableMirror import DeformableMirror

# Specifying a given number of actuators along the diameter:
nAct = 16       # This value is the number of actuator in the pupil! So 1 extra actuator is added automatically

dm = DeformableMirror(telescope  = tel,                        # Telescope
                    nSubap       = nAct,                       # Number of subaperture of the system considered (by default the DM has n_subaperture + 1 actuators to be in a Fried Geometry)
                    mechCoupling = 0.35,                       # Mechanical Coupling for the influence functions
                    coordinates  = None,                       # Coordinates in [m]. Should be input as an array of size [n_actuators, 2]
                    pitch        = tel.D/nAct)                 # Inter actuator distance. Only used to compute the influence function coupling. The default is based on the n_subaperture value.


# Plot the dm actuators coordinates with respect to the pupil
plt.figure()
plt.imshow(np.reshape(np.sum(dm.modes**7,axis=1),[tel.resolution,tel.resolution]).T + tel.pupil,extent=[-tel.D/2,tel.D/2,-tel.D/2,tel.D/2])
plt.plot(dm.coordinates[:,0],dm.coordinates[:,1],'rx')
plt.xlabel('[m]')
plt.ylabel('[m]')
plt.title('DM Actuator Coordinates VS Telescope Pupil')

In [ ]:
# Apply a command on the DM and propagate light through it
dm.coefs = np.random.randn(dm.nValidAct)*300e-9 # Random vector for the valid actuators

# Propagate through the DM
tel.resetOPD()              # Reset to remove any previous optical path
ngs*tel*dm

plt.figure()
plt.imshow(dm.OPD*1e9)
plt.title('OPD DM [nm]')
plt.colorbar()

plt.figure()
plt.imshow(tel.OPD*1e9)
plt.title('OPD Telescope [nm]')
plt.colorbar()



# Pyramid wavefront sensor

In [ ]:
from OOPAO.Pyramid import Pyramid

# Make sure tel and atm are separated to initialize the PWFS
tel.isPaired = False
tel.resetOPD()

wfs = Pyramid(nSubap            = n_subaperture,                # Number of subaperture = number of pixel accros the pupil diameter
              telescope         = tel,                          # Telescope object
              lightRatio        = 0.5,                          # Flux threshold to select valid sub-subaperture
              modulation        = 3,                            # Tip tilt modulation radius
              n_pix_separation  = 4,                            # Number of pixel separating the different pupils
              n_pix_edge        = 2,                            # Number of pixel on the edges of the pupils
              postProcessing    = 'fullFrame_camera_flux')      # PWFS processing method

# Propagate the light to the Wave-Front Sensor
tel*wfs

plt.close('all')
plt.figure()
plt.imshow(wfs.cam.frame)
plt.title('WFS Camera Frame')
plt.colorbar()

plt.figure()
plt.imshow(wfs.validSignal)
plt.title('WFS Valid Signal')
plt.colorbar()


In [ ]:
# Applying noise to the WFS
# Photon noise is either on or off. Read noise is specified in electrons
wfs.cam.photonNoise = True
wfs.cam.readNoise = 1
ngs*tel*wfs

plt.figure()
plt.imshow(wfs.cam.frame); plt.colorbar()
plt.title('WFS Camera Frame - With Noise')

wfs.cam.photonNoise = False
wfs.cam.readNoise = 0
ngs*tel*wfs
plt.figure()
plt.imshow(wfs.cam.frame); plt.colorbar()
plt.title('WFS Camera Frame - Without Noise')

# Modal basis - KL modes

In [ ]:
from OOPAO.calibration.compute_KL_modal_basis import compute_KL_basis
# Use the default definition of the KL modes with forced Tip and Tilt. For more complex KL modes, consider the use of the compute_KL_basis function.
M2C_KL = compute_KL_basis(tel, atm, dm,lim = 1e-2) # matrix to apply modes on the DM

# Apply the 10 first KL modes
dm.coefs = M2C_KL[:,:10]
# Propagate through the DM
ngs*tel*dm
# Show the first 10 KL modes applied on the DM
displayMap(tel.OPD)

# Calibration

In [ ]:
from OOPAO.calibration.InteractionMatrix import InteractionMatrix

# Amplitude of the modes in m
stroke=ngs.wavelength/16

# Number of modes to control
nModes = 160

tel-atm
# Zonal interaction matrix
calib = InteractionMatrix(ngs            = ngs,
                          atm            = atm,
                          tel            = tel,
                          dm             = dm,
                          wfs            = wfs,
                          M2C            = M2C_KL[:,:nModes],    # M2C matrix used
                          stroke         = stroke,    # Stroke for the push/pull in M2C units
                          nMeasurements  = 6,         # Number of simultaneous measurements
                          noise          = 'off',     # Disable wfs.cam noise
                          display        = True,      # Display the time using tqdm
                          single_pass    = False)      # Only push to compute the interaction matrix instead of push-pull



In [ ]:
# Plot singular values and display WFS signals
plt.figure()
plt.plot(calib.eigenValues)
plt.xlabel('Modes')
plt.ylabel('Singular values')
print('Conditioning number: '+str(calib.cond))

# Closed loop

In [ ]:
# Closed loop parameters
loopGain = 0.5
nIter = 100           # Number of loop iterations

# Reconstructor
reconstructor = M2C_KL[:,:nModes]@calib.M

# Set noise and guide star magnitude
wfs.cam.photonNoise = True
wfs.cam.readNoise = 1
ngs.magnitude = 5

# Allocate memory to save data
strehl = np.zeros(nIter)          # Strehl ratio
turb_rms = np.zeros(nIter)        # Turbulence rms
res_rms = np.zeros(nIter)    # Residual rms


In [ ]:
# Closed loop

# Reset telescope, atmosphere and DM
tel.resetOPD()
dm.coefs=0
ngs*tel*dm*wfs
atm.generateNewPhaseScreen(seed = 10)
tel+atm
wfsSignal = 0*wfs.signal;

for n in range(0,nIter):

    # Update atmosphere
    atm.update()

    # Save turbulent wavefront and PSF
    turb_WF = tel.mean_removed_OPD.copy()
    turb_PSF = tel.computePSF(zeroPaddingFactor = 6)
    turb_rms[n]=np.sqrt(np.mean(turb_WF[np.where(tel.pupil>0)]**2))*1e9

    # Propagate light from the NGS through the atmosphere, telescope, DM to the WFS
    atm*ngs*tel*dm*wfs

    # Save residual cwavefront and PSF
    res_WF = tel.mean_removed_OPD.copy()
    res_PSF = tel.computePSF(zeroPaddingFactor = 6)
    res_rms[n]=np.sqrt(np.mean(res_WF[np.where(tel.pupil>0)]**2))*1e9

    # Reconstruct and update DM commands
    dm.coefs = dm.coefs-loopGain*(reconstructor@wfsSignal)

    # Store WFS signals (for 2 frame delay)
    wfsSignal = wfs.signal.copy()

    print('Loop '+str(n)+'/'+str(nIter)+' -- turbulence: '+str(turb_rms[n])+' -- residual:' +str(res_rms[n]))


# Plot results
plt.figure()
plt.plot(turb_rms)
plt.plot(res_rms)
plt.legend(['turbulence','residual'])
plt.xlabel('Loop iteration')
plt.ylabel('Wavefront rms [nm]')

